In [16]:
from transformers import AutoTokenizer, AutoModel,
import torch
import math
from transformer_lens import HookedTransformer

In [2]:
def normalize(R):
    with torch.no_grad():
        mean = R.mean(dim=0)
        R = R - mean
        norms = torch.norm(R, p=2, dim=1, keepdim=True)
        R = R/norms
    return R

def cal_cov(R):
    with torch.no_grad():
        Z = torch.nn.functional.normalize(R, dim=1)
        A = torch.matmul(Z.T, Z)/Z.shape[0]
    return A

def cal_entropy(A):
    with torch.no_grad():
        eig_val = torch.svd(A / torch.trace(A))[1] 
        entropy = - (eig_val * torch.log(eig_val)).nansum().item()
        normalized_entropy = entropy/math.log(A.shape[0])
    return normalized_entropy


In [25]:
def hook_fn(module, input, output):

    normal = normalize(output)
    cov = cal_cov(normal)
    entropy_value = cal_entropy(cov)
    print(entropy_value)


In [11]:
prompt = "good job"

In [18]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
MODEL: HookedTransformer = HookedTransformer.from_pretrained("gpt2")

input_ids = tokenizer.encode(prompt, return_tensors="pt")

Loaded pretrained model gpt2 into HookedTransformer


In [20]:
d_model: int = MODEL.cfg.d_model
n_layers: int = MODEL.cfg.n_layers
n_heads: int = MODEL.cfg.n_heads
d_vocab: int = MODEL.cfg.d_vocab
print(f"{d_model=}, {n_layers=}, {n_heads=}, {d_vocab=}")

d_model=768, n_layers=12, n_heads=12, d_vocab=50257


In [27]:
layer_to_hook = 6  
hook_handle = list(MODEL.children())[layer_to_hook].register_forward_hook(hook_fn)

In [28]:
with torch.no_grad():
    MODEL(input_ids)


/var/folders/08/h4ttblxd6m5fjbzyhmxs4bk00000gn/T/ipykernel_72611/778970859.py:12: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3618.)
  A = torch.matmul(Z.T, Z)/Z.shape[0]


RuntimeError: Expected size for first two dimensions of batch2 tensor to be: [50257, 1] but got: [50257, 2].